# Evolutionary games in Python

In [2]:
import numpy as np

## Playing a fixed number of games

In [ ]:
def play_round(game, p0, p1, actions_hist):
    """
    Plays one round of a game whose payoff matrix is `game`,
    with two players whose strategies are the functions p1 and p2,
    with a past `history` of the players' actions, which are elements
    of {0, ..., n-1}, n being the number of rows of `game`.

    Payoff matrix `game`:
    rows: player actions
    columns: opponent actions
    actions: 0 = cooperate, 1 = defect
    """
    # each player's action
    p0_action = p0(actions_hist, player=0)
    p1_action = p1(actions_hist, player=1)

    # each player's payoff
    p0_payoff = game[p0_action, p1_action]
    p1_payoff = game[p1_action, p0_action]
    
    return (p0_action, p1_action), (p0_payoff, p1_payoff)


def play(game, p0, p1, actions_hist, n=1):
    """
    Play `n` rounds of `game` with players `p0` and `p1` starting
    from `init_actions`.
    """
    if len(actions_hist) == 0:
        raise ValueError("There must be at least one initial pair of actions")
        
    init_actions = actions_hist[0]
    payoffs_hist = [(game[init_actions], game[tuple(reversed(init_actions))])]
    
    for i in range(1, n):
        a, p = play_round(game, p0, p1, actions_hist)
        payoffs_hist.append(p)
        actions_hist.append(a)
        
    return actions_hist, payoffs_hist

### Some strategies

In [87]:
def always_defect(actions_hist, player=0):
    """
    The strategy that always defects no matter what.
    """
    return 1

def always_cooperate(actions_hist, player=0):
    """
    The strategy that always defects no matter what.
    """
    return 0

def tit_for_tat(actions_hist, player=0):
    """
    Plays the opponent's last move.
    """
    last_play = actions_hist[-1]
    
    return last_play[not player]

def tit_for_two_tats(actions_hist, player=0):
    """
    Defect if the opponent defects twice in a row,
    otherwise cooperate.
    """
    last_play = actions_hist[-1]
    second_to_last_play = actions_hist[-2]
    
    return last_play[not player] and second_to_last_play[not player]

def grudger(actions_hist, player=0):
    """
    Cooperates until the first defection of the opponent,
    then defects forever.
    """
    nplays = len(actions_hist)
    opponent_hist = [actions_hist[i][not player] for i in range(nplays)]
    
    return int(1 in opponent_hist)

def random(history, player=0):
    """
    Fair coin flip.
    """
    return np.random.default_rng().choice([0,1])

### Testing it out

In [88]:
# a payoff matrix for the prisoner's dilemma
prisoners_dilemma = np.array([[2, 0],
                              [3, 1]])

# both players cooperate initially
init_actions = [(0, 0)]

actions, payoffs = play(
    game = prisoners_dilemma,
    p0 = tit_for_tat,
    p1 = random,
    actions_hist = init_actions,
    n = 10
)
print('actions (p0, p1): ', actions)
print('payoffs (p0, p1): ', payoffs)

actions (p0, p1):  [(0, 0), (0, 0), (0, 1), (1, 1), (1, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]
payoffs (p0, p1):  [(2, 2), (2, 2), (0, 3), (1, 1), (3, 0), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2)]


## Tournaments

In [ ]:
def round_robin(players):
    """
    A round-robin tournament (all-to-all) between a list of players.
    """

    
